In [1]:
import numpy as np
import pandas as pd
from copy import copy
from pathlib import Path
from itertools import chain
from tqdm.notebook import tqdm

import data_tools.graphs as gt
from hetnet_ml.extractor import MatrixFormattedGraph, piecewise_extraction

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
nodes = pd.read_csv('../2_pipeline/13a_Model_Prep_Holdout_Set/out/nodes.csv', dtype=str)
edges = pd.read_csv('../2_pipeline/13a_Model_Prep_Holdout_Set/out/edges.csv', dtype=str)

mg = MatrixFormattedGraph(nodes, edges, 'ChemicalSubstance', 'Disease', max_length=4, w=0.4, n_jobs=30)

Processing node and edge data...
Initializing metagraph...
Generating adjacency matrices...


100%|██████████| 68/68 [00:56<00:00,  1.21it/s]



Determining degrees for each node and metaedge


100%|██████████| 68/68 [00:27<00:00,  2.49it/s]



Weighting matrices by degree with dampening factor 0.4...


100%|██████████| 68/68 [00:01<00:00, 55.86it/s]


## Strange behavior

For whatever reason, the imports below result in the above initizalizaiton of mg not finishing, so we've loaded mg first, then will continue the imports

In [3]:
from scipy.sparse import issparse, csc_matrix, csr_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel

from xgboost import XGBClassifier

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Data Transformer

We use an inverse hyperbolic sine transformation to transform the features.

The transormation is thus:

$$sinh^{-1} \left( \frac{X_{mp}}{\sigma_{mp}} \right)$$

where $X_{mp}$ is the column in the dwpc feature matrix $X$ coreesponding to metapath $mp$ and $\sigma_{mp}$ is the mean of the column.

In [4]:
class MeanScaledArcsinhTransformer(TransformerMixin):
        
    def fit(self, X, y=None):
        if issparse(X):
            self.initial_mean_ = X.tocoo().tocsc().mean(axis=0).A[0]
        else:
            self.initial_mean_ = X.mean(axis=0)

        # If input was DataFrame, Converts resultant series to ndarray
        try:
            self.initial_mean_ = self.initial_mean_.values
        except:
            pass
        
        # If inital mean == 0, likely all values were zero
        # this prevents issues later.
        self.initial_mean_[np.where(self.initial_mean_ == 0.0)] = 1
        
        return self

    def transform(self, X, y=None):
        if issparse(X):
            return np.arcsinh(X.tocoo().tocsc().multiply(self.initial_mean_**-1))
        return np.arcsinh(X / self.initial_mean_)


# Homegrown Feature Selector

Runs 6 analysis on the traning data to select features.

1. Correlation to the output
2. Chi_squared test
3. Recurssive Feature Elimantion on a Ridge Regressor
4. Embedded Feature Selection from a Lasso Regressor
5. Embedded Feature Selection from a Randomn Forest Classifier
6. Embedded Feature Selection from a Gradient Boosting Classifier

Each analysis will select `num_feats` best features. The selected features will then by chosen via a voting method with `min_selections` out of the 6 elements required to for a feature to be kept. 


We have also added an option for `always_keep`:  This allows for domain expertise to be factored into the feature selection process.  In our case, we know some metapaths are specifically mechanistic, so we want to include those wherever possible

In [5]:
def cor_selector(X, y, feature_names, num_feats):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in range(X.shape[1]):
        if issparse(X):
            x = X[:, i].A.reshape(len(y))
        else:
            x = X[:, i]
        cor = np.corrcoef(x, y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = np.array(feature_names)[np.argsort(np.abs(cor_list))[-num_feats:].tolist()].tolist()    
    # feature selection? 0 for not select, 1 for select
    return [True if i in cor_feature else False for i in feature_names]

def chi2_selector(X, y, num_feats):
    this_selector = SelectKBest(chi2, k=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()

def rfe_selector(X, y, num_feats, random_state=None):
    this_selector = RFE(estimator=LogisticRegression(C=.1, solver='liblinear', random_state=random_state), 
                        n_features_to_select=num_feats, step=.2, verbose=5)
    this_selector.fit(X, y)
    return this_selector.get_support()

def embeded_lr_selector(X, y, num_feats, random_state=None):
    this_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear', random_state=random_state), 
                                    max_features=num_feats)
    this_selector.fit(X, y)

    return this_selector.get_support()

def embeded_rf_selector(X, y, num_feats, n_jobs, random_state=None):
    rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=n_jobs, random_state=random_state)
    this_selector = SelectFromModel(rfc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()

def embeded_xgb_selector(X, y, num_feats, n_jobs=1, random_state=None):
    # XGBoost takes 0 as default random state
    if random_state is None:
        random_state = 0
    # Paramaters optimized for speed, rather than accuracy (as we have 5 other estimators also providing votes)
    xgbc = XGBClassifier(max_depth=5, n_estimators=200, learning_rate=.16, min_child_weight=1, colsample_bytree=.8,
                         n_jobs=n_jobs, random_state=random_state)
    this_selector = SelectFromModel(xgbc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()


class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_features=100, min_selections=4, n_jobs=1, feature_names=None, always_keep=None,
                 random_state=None):
        self.num_features = num_features
        self.min_selections = min_selections
        self.n_jobs = n_jobs
        self.feature_names = feature_names
        self.always_keep = always_keep
        self.random_state = random_state
        
    def fit(self, X, y=None):
                
        X_norm = MaxAbsScaler().fit_transform(X)
        if issparse(X):
            if type(X) != csc_matrix:
                X = X.tocsc()
            X_norm = X_norm.tocsc()
        
        print('Running Cor')
        cor_support = cor_selector(X, y, self.feature_names, self.num_features)
        print('Running Chi2')
        chi_support = chi2_selector(X_norm, y, self.num_features)
        print('Running RFE')
        rfe_support = rfe_selector(X_norm, y, self.num_features, self.random_state)
        print('Running LR')
        embeded_lr_support = embeded_lr_selector(X_norm, y, self.num_features, self.random_state)
        print('Running RF')
        embeded_rf_support = embeded_rf_selector(X, y, self.num_features, 
                                                 n_jobs=self.n_jobs, random_state=self.random_state)
        print('Running XG')
        embeded_xgb_support = embeded_xgb_selector(X, y, self.num_features, 
                                                   n_jobs=self.n_jobs, random_state=self.random_state)
        
        feature_selection_df = pd.DataFrame({'feature':self.feature_names, 'pearson':cor_support, 'chi_2':chi_support, 
                                             'rfe':rfe_support, 'logistics':embeded_lr_support,
                                             'random_forest':embeded_rf_support, 'xgboost':embeded_xgb_support})  

        feature_selection_df['total'] = np.sum(feature_selection_df, axis=1)
        self.feature_selection_df_ = feature_selection_df
        
        keep_features = feature_selection_df.query('total >= {}'.format(self.min_selections))['feature'].tolist()
        
        # Keep the features that we always want (e.g. domain expertise)
        if self.always_keep is not None:
            keep_features.extend(self.always_keep)
        
        self.keep_features_ = [f for f in self.feature_names if f in keep_features]
        
        return self
    
    def transform(self, X, y=None):
        
        if issparse(X) and type(X) != csc_matrix:
            X = X.tocsc()
        return X[:, [i for i, f in enumerate(self.feature_names) if f in self.keep_features_]]

In [6]:
def sparse_std(data, axis=1):                                                                              
    """take the standard deviation of a sparse matrix"""                  

    def get_vec_std(vec):                                                                                       
        return vec.A.std(ddof=1)                                                                                

    stds = []        
    
    # ensure the correct matrix type for easy row or column subsetting
    if axis==1 and type(data) != csc_matrix:
        data = data.tocoo().tocsc()
    if axis==0 and type(data) != csr_matrix:
        data = data.tocoo().tocsr()
    
    # Get the std for each vector along the given axis individually
    for i in range(data.shape[axis]):                                                                              
        if axis==1:
            stds.append(get_vec_std(data.getcol(i)))       
        elif axis==0:
            stds.append(get_vec_std(data.getrow(i)))       
        
    return np.array(stds)

In [7]:
def get_model_coefs(model, X, f_names):                                                                                 
    """Helper Function to quickly return the model coefs and correspoding fetaure names"""                              
                                                                                                                        
    # Ensure we have a numpy array for the features                                                                     
    if type(X) == pd.DataFrame:                                                                                         
        X = X.values
        
                                                                                                                        
    # Grab the coeffiencts                                                                                              
    coef = model.coef_                                                                                                  
    # Some models return a double dimension array, others only a single                                                 
    if len(coef) != len(f_names):                                                                                       
        coef = coef[0]                                                                                                  
                                                                                                                        
    # insert the intercept                                                                                              
    coef = np.insert(coef, 0, model.intercept_)                                                                         
    names = np.insert(f_names, 0, 'intercept')                                                                          
                                                                                                                        
    # Calculate z-score scaled coefficients based on the features                                                       
    if issparse(X):
        if type(X) != csc_matrix:
            X = X.tocoo().tocsc()
        z_intercept = coef[0] + sum(coef[1:] * X.mean(axis=0).A[0])
        z_coef = coef[1:] * sparse_std(X, axis=1)
        z_coef = np.insert(z_coef, 0, z_intercept)
    else:
        z_intercept = coef[0] + sum(coef[1:] * X.mean(axis=0))                                                              
        z_coef = coef[1:] * X.std(axis=0)                                                                                   
        z_coef = np.insert(z_coef, 0, z_intercept)                                                                          
                                                                                                                        
    # Return                                                                                                            
    return pd.DataFrame([names, coef, z_coef]).T.rename(columns={0:'feature', 1:'coef', 2:'zcoef'})    

In [8]:
float_size = 64 # bits
bits_per_gb = 8589934592

def print_mem_info(n_comp, n_dis, n_mps):
    print("{:,} Compounds * {:,} Diseases = {:,} C-D Pairs".format(n_comp, n_dis,
                                                                   n_comp * n_dis))
    print("{:,} C-D Pairs * {:,} Metapaths = {:,} Matrix Values".format(n_comp * n_dis,
                                                                    n_mps, 
                                                                    n_comp * n_dis * n_mps))

    print('{:1,.1f} GB of matrix values'.format(n_comp * n_dis * n_mps * float_size / (bits_per_gb)))
    
    print('{:1,.3f} GB per metapath'.format(n_comp * n_dis * float_size / (bits_per_gb)))

In [9]:
train_frac = 0.15
rs = 20200123


treat_comps = set(edges.query('type == "treats_CtD"')['start_id'])
# Sample the negatives and subsample 
keep_comps = set(nodes.query('id not in @treat_comps and label == "ChemicalSubstance"')
                      .sample(frac=train_frac*.01, random_state=rs)['id'])
# Then subsample the positives
keep_comps = keep_comps | set(nodes.query('id in @treat_comps')
                                   .sample(frac=train_frac, random_state=rs+1)['id'])

treat_dis = set(edges.query('type == "treats_CtD"')['end_id'])
# Sample the negatives and subsample cv
keep_dis = set(nodes.query('label == "Disease" and id not in @treat_dis')
                    .sample(frac=train_frac*.01, random_state=rs+2)['id'])
# Take the diseases Treated by these compounds
keep_dis = keep_dis | set(edges.query('type == "treats_CtD" and start_id in @keep_comps')['end_id'])

In [10]:
# MP counts will be the same with or without weights, so just use the original
all_mp_counts = pd.read_csv('../2_pipeline/13b_Model_Prep_Metapath_Membership_Analysis/out/all_mp_counts.csv')
all_mp_counts.head(2)

,mp,pair_count,subset,frac,sim_mp
0,CpoAawD,5854758,all_pairs,0.010287,False
1,CpoAsoD,0,all_pairs,0.000000,False


In [11]:
non_sim_names = all_mp_counts.query('sim_mp == False')['mp'].unique().tolist()
mp_qr = all_mp_counts.query('subset == "all_pairs" and mp in @non_sim_names and pair_count > 0')
good_mps = mp_qr['mp'].tolist()

In [12]:
print_mem_info(len(keep_comps), len(keep_dis), len(good_mps))

1,404 Compounds * 2,906 Diseases = 4,080,024 C-D Pairs
4,080,024 C-D Pairs * 7,012 Metapaths = 28,609,128,288 Matrix Values
213.2 GB of matrix values
0.030 GB per metapath


In [13]:
print('{:,} Positive training examples in subset'.format(len(edges.query('start_id in @keep_comps and type == "treats_CtD"'))))

8,607 Positive training examples in subset


### Bring in the known metapaths found in DrugMechDB

In [14]:
mech_mps = pd.read_csv('../0_data/manual/mech_mps.txt', header=None)[0].values
dmdb_feat = set(all_mp_counts.query('mp in @mech_mps and subset == "all_pairs" and pair_count > 0 and sim_mp == False')['mp'])

len(dmdb_feat)

25

# Prepare output location

In [15]:
this_name = '13c_Model_Prep_Hyperparam_tuning'

out_dir = Path('../2_pipeline').joinpath(this_name, 'out').resolve()
tmp_dir = out_dir.parent.joinpath('tmp')

out_dir.mkdir(parents=True, exist_ok=True)
tmp_dir.mkdir(parents=True, exist_ok=True)

# Extract the features one time

Use the pair counts to sort metapaths extaction as a niave load balancer

In [16]:
def sort_mps_for_pw_extraction(n_big_calcs, big_per_block, mp_list, frac_info):
    
    big_mp = frac_info.sort_values('frac', ascending=False).head(n_big_calcs)['mp'].tolist()
    other_mp = list(set(mp_list) - set(big_mp))

    block_size = len(other_mp) // (len(big_mp) // big_per_block)
    n_blocks = ((len(big_mp) + len(other_mp)) // block_size)

    out = []
    for i in range(n_blocks):
        for j in range(big_per_block):
            idx = i*big_per_block + j
            out.append(big_mp[idx])
        out += other_mp[i*block_size:(i+1)*block_size] 
        
    out += list(set(other_mp) - set(out))    
    
    return out, block_size

In [17]:
# 30, 5 finishes < 30 min. <- OVERFLOW
# 40, 8 finishes 18min 46s <- OVERFLOW
to_xtract, block_size = sort_mps_for_pw_extraction(100, 5, good_mps, mp_qr)

In [18]:
len(to_xtract), block_size

(7012, 345)

In [19]:
%%time
## Extract the metapaths to do some prep work
(pairs, feats), test_dwpc = piecewise_extraction(function=mg.extract_dwpc, 
                                 to_split='metapaths', block_size=block_size,
                                 axis=1,
                                 metapaths=to_xtract, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_dis),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)

100%|██████████| 21/21 [44:59<00:00, 128.53s/it]


CPU times: user 2min 52s, sys: 1min 24s, total: 4min 17s
Wall time: 45min 2s


# Training Targets for feature selection¶

In [20]:
pos_tups = edges.query('type == "treats_CtD"')[['start_id', 'end_id']].apply(tuple, axis=1).tolist()
pos_tups[:2]

[('CHEBI:100147', 'DOID:0050400'), ('CHEBI:100147', 'DOID:13148')]

In [21]:
pos_tups = set(pos_tups)

In [22]:
y = []
for row in tqdm(pairs.itertuples(), total=len(pairs)):
    if set([(row.chemicalsubstance_id, row.disease_id)]) & pos_tups:
        y.append(1)
    else:
        y.append(0)

y = np.array(y)
        
print(len(pairs), len(y), sum(y))


4080024 4080024 8607


In [23]:
pairs['status'] = y

# Subsample the potnetial training examples

Our class is so imbalanced, to get a sizeable number of positive training examples, we end up with many orders of magnitude more negative examples.  Many of those examples will have no connections from the compound to the disease of interest, this providing a zero row in the matrix. We will not waste time training on those values, and instead focus on the ones that distinguish the positive from the negative training examples.

As this is just hyperparameter tunings, to speed things up, we will also limit the positive examples to a small portion of the negative examples, 100x (2 orders of magnitude) larger than the number of postitives. 

In [24]:
# Get the rows that are Non-zero
nz_index = pairs[test_dwpc.getnnz(1)>0].index

# have the number of postivies to get 100x this for the negatives.
n_pos = pairs['status'].sum()

# Sample the nonzero negative examples at a rate of 100x the positive samples
neg_index = pairs.loc[nz_index].query('status == 0').sample(n=100*n_pos, random_state=rs+10).sort_index().index

# and of course take the training postivies
pos_index = pairs.query('status == 1').index

# Union the two
train_index = pos_index.union(neg_index)

In [25]:
feats = np.array(feats)
nz_feats = feats[test_dwpc.getnnz(0)>0]
feat_index = test_dwpc.getnnz(0)>0

In [26]:
# Get our compounds as ndarrays for easy indexing with sklearns StratifiedKFold
keep_comps = np.array(list(keep_comps))
# Need to know how to properly stratify the split
is_treat_comp = np.array([1 if c in treat_comps else 0 for c in keep_comps])

## Select the features

This is a time consuming and costly step. We will do once with the initial DWPC for parameter tuning. We will perform again at the end with the selected parameters.

In [27]:
%%time
msat = MeanScaledArcsinhTransformer()
trans_dwpc = msat.fit_transform(test_dwpc[train_index][:, feat_index])

CPU times: user 35.2 s, sys: 6.47 s, total: 41.7 s
Wall time: 41.7 s


In [28]:
%%time

if tmp_dir.joinpath('test_feats.txt').exists():
    keep_features = pd.read_csv(tmp_dir.joinpath('test_feats.txt'))['0'].tolist()
else:
    fsel = FeatureSelector(num_features=500, min_selections=4, n_jobs=30, 
                           feature_names=nz_feats.tolist(), always_keep=dmdb_feat, random_state=rs+5)
    sel_dwpc = fsel.fit_transform(trans_dwpc, y[train_index])
    keep_features = fsel.keep_features_
    pd.Series(keep_features).to_csv(tmp_dir.joinpath('test_feats.txt'), index=False)
    fsel.feature_selection_df_.to_csv(tmp_dir.joinpath('test_fs_df.csv'), index=False)

Running Cor


/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Running Chi2
Running RFE
Fitting estimator with 5913 features.
Fitting estimator with 4731 features.
Fitting estimator with 3549 features.
Fitting estimator with 2367 features.
Fitting estimator with 1185 features.
Running LR
Running RF
Running XG
CPU times: user 4h 10min 32s, sys: 1min 26s, total: 4h 11min 59s
Wall time: 30min 29s


## Prep for Hyperparam optimization

In [29]:
all_dwpc = dict()

def get_dwpc(w):
    global all_dwpc
    
    # only extract if not previously extracted
    if w not in all_dwpc.keys():
        mg.update_w(w)
        (pairs, feats), dwpc = mg.extract_dwpc(metapaths=keep_features, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_dis),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)
        # next step is split, so need sparse rows
        dwpc = dwpc.tocoo().tocsr()
        #all_dwpc[w] = dwpc
        return dwpc
    else:
        return all_dwpc[w]

In [30]:
def neg_log_scal(val):
    """Scale values logrithmicly. Larger input yields smaller result"""
    return (1-np.log1p(val)) 

def pos_log_scal(val):
    """Scale values logrithmicly. Larger input yields larger result"""
    return np.log1p(val)


def auroc_mean_loss(auroc_mean, strength=2, scal_factor=.5):
    mean_roc_shift =  (np.abs(auroc_mean - 0.5)) / scal_factor
    baseline_roc = neg_log_scal(1)
    return (neg_log_scal(mean_roc_shift) - baseline_roc) * strength

def avg_pre_mean_loss(avg_pre_mean, strength=2.5, scal_factor=.15):
    avg_pre_mean_scal = avg_pre_mean / scal_factor
    baseline_prc = neg_log_scal(1)
    return (neg_log_scal(avg_pre_mean_scal) - baseline_prc) * strength

def auroc_std_loss(auroc_std, strength=2):
    return pos_log_scal(auroc_std) * strength

def avg_pre_std_loss(avg_pre_std, strength=2):
    return pos_log_scal(avg_pre_std) * strength

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std):
    mean_roc_loss = auroc_mean_loss(auroc_mean)
    mean_pre_loss = avg_pre_mean_loss(avg_pre_mean)
    std_roc_loss = auroc_std_loss(auroc_std)
    std_pre_loss = avg_pre_std_loss(avg_pre_std)
    
    return sigmoid(mean_roc_loss + mean_pre_loss + std_roc_loss + std_pre_loss)-1
    
    

In [31]:
import pickle
from time import time

def hyperopt(param_space, y, num_eval):

    def objective_function(params):
        dwpc_params = {k.split('__')[1]: v for k, v in params.items() if k.split('__')[0] == 'dwpc'}
        enet_params = {k.split('__')[1]: v for k, v in params.items() if k.split('__')[0] == 'enet'}

        this_w = dwpc_params['w']

        # Set up the post feature extraction pipeline
        post_extraction_pipeline = Pipeline(
            [('transformer', MeanScaledArcsinhTransformer()),
             ('maxabs_scale', MaxAbsScaler()),
             ('e_net', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100, **enet_params,
                                          random_state=rs+6))], verbose=True)

        ## Get the dwpc information for the current pairs
        dwpc = get_dwpc(this_w)
        this_dwpc = dwpc[train_index]

        cv = cross_validate(post_extraction_pipeline, this_dwpc, y, cv=5, 
                            scoring=['average_precision', 'roc_auc'], return_estimator=True)

        # Write out scores for each run
        with open(tmp_dir.joinpath('scores_w_{0:1.4f}_C_{1:1.5f}_l1_{2:1.4f}.txt'.format(
                                   this_w, enet_params['C'], enet_params['l1_ratio'])), 'w') as f_out:

            f_out.write(', '.join([str(s) for s in cv['test_average_precision']]))
            f_out.write('\n')
            f_out.write(', '.join([str(s) for s in cv['test_roc_auc']]))
            f_out.write('\n')

        auroc_mean = cv['test_roc_auc'].mean()
        avg_pre_mean = cv['test_average_precision'].mean()
        auroc_std = cv['test_roc_auc'].std()
        avg_pre_std = cv['test_average_precision'].std()
        
        score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)
        
        print('Mean AUROC: {:1.4f}'.format(auroc_mean))
        print('Mean Avg Pre: {:1.4f}'.format(avg_pre_mean))
        print('STD AUROC: {:1.4f}'.format(auroc_std))
        print('STD Avg Pre: {:1.4f}'.format(avg_pre_std))
        print('Loss: {:1.4f}'.format(score))
        
        return {'loss': score, 'status': STATUS_OK}

    start = time()
   
    
    trials = Trials()
    best_param = fmin(objective_function, 
                      param_space, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate= np.random.RandomState(1))
    
    print(time() - start)
    return trials, best_param
    

In [32]:
# Previous best before adding new keep_features: 
# '1l_ratio': 0.10455936193818496, 'C': 0.000556880900960339, 'w': 0.2640929485381926
param_hyperopt = {
    'dwpc__w': hp.uniform('w', 0.01, 1),
    'enet__C': hp.loguniform('C', np.log(0.001), np.log(1)),
    'enet__l1_ratio': hp.uniform('l1_ratio', .01, .99),
}

In [33]:
trials, best_param = hyperopt(param_hyperopt, y[train_index], 50)

Changing w from 0.4 to 0.2471551211320533. Please wait...
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/68 [00:00<?, ?it/s]

  3%|2         | 2/68 [00:00<00:03, 17.06it/s]

 15%|#4        | 10/68 [00:00<00:02, 22.02it/s]

 41%|####1     | 28/68 [00:00<00:01, 29.87it/s]

 53%|#####2    | 36/68 [00:00<00:00, 35.54it/s]

 68%|######7   | 46/68 [00:00<00:00, 43.95it/s]

 88%|########8 | 60/68 [00:00<00:00, 55.24it/s]

100%|##########| 68/68 [00:00<00:00, 69.89it/s]


Preparing function arguments...                       
Calculating DWPCs...                                  
  0%|          | 0/50 [00:07<?, ?trial/s, best loss=?]

  0%|          | 0/139 [00:00<?, ?it/s]

  1%|1         | 2/139 [00:00<00:08, 16.23it/s]

  2%|2         | 3/139 [00:00<00:10, 12.74it/s]

  7%|7         | 10/139 [00:00<00:08, 15.98it/s]

 10%|#         | 14/139 [00:00<00:06, 18.94it/s]

 14%|#4        | 20/139 [00:00<00:05, 23.44it/s]

 17%|#7        | 24/139 [00:00<00:04, 26.54it/s]

 20%|##        | 28/139 [00:00<00:04, 27.23it/s]

 24%|##4       | 34/139 [00:01<00:03, 30.51it/s]

 27%|##7       | 38/139 [00:01<00:03, 32.35it/s]

 30%|###       | 42/139 [00:01<00:02, 32.41it/s]

 33%|###3      | 46/139 [00:01<00:02, 31.74it/s]

 39%|###8      | 54/139 [00:01<00:02, 32.63it/s]

 42%|####2     | 59/139 [00:01<00:02, 33.73it/s]

 46%|####6     | 64/139 [00:01<00:02, 35.90it/s]

 51%|#####1    | 71/139 [00:01<00:01, 41.92it/s]

 55%|#####4    | 76/139 [00:02<00:01, 40.45it/s]

 60%|#####9    | 83/139 [00:02<00:01, 46.22it/s]

 64%|######4   | 89/139 [00:02<00:01, 46.89it/s]

 68%|######8   | 95/139 [00:02<00:01, 42.07it/s]

 73%|######

                                                      
Reshaping Result Matrices...
  0%|          | 0/50 [02:47<?, ?trial/s, best loss=?]

...

Preparing function arguments...                                                       
Calculating DWPCs...                                                                  
 98%|█████████▊| 49/50 [7:35:20<07:37, 457.80s/trial, best loss: -0.18523043825516827]

  0%|          | 0/139 [00:00<?, ?it/s]

  1%|1         | 2/139 [00:00<00:13, 10.20it/s]

  7%|7         | 10/139 [00:00<00:09, 13.29it/s]

 11%|#         | 15/139 [00:00<00:08, 14.10it/s]

 14%|#4        | 20/139 [00:00<00:06, 17.49it/s]

 17%|#6        | 23/139 [00:00<00:06, 19.01it/s]

 20%|##        | 28/139 [00:01<00:04, 23.22it/s]

 24%|##4       | 34/139 [00:01<00:03, 28.35it/s]

 28%|##8       | 39/139 [00:01<00:03, 31.64it/s]

 33%|###3      | 46/139 [00:01<00:02, 37.31it/s]

 37%|###7      | 52/139 [00:01<00:02, 41.82it/s]

 42%|####1     | 58/139 [00:01<00:01, 44.49it/s]

 46%|####6     | 64/139 [00:01<00:01, 47.06it/s]

 50%|#####     | 70/139 [00:01<00:01, 37.12it/s]

 55%|#####4    | 76/139 [00:02<00:01, 39.72it/s]

 62%|######1   | 86/139 [00:02<00:01, 44.51it/s]

 67%|######6   | 93/139 [00:02<00:00, 48.42it/s]

 71%|#######1  | 99/139 [00:02<00:00, 48.61it/s]

 76%|#######5  | 105/139 [00:02<00:00, 40.63it/s]

 81%|########1 | 113/139 [00:02<00:00, 47.58it/s]

 86%|###

                                                                                      
Reshaping Result Matrices...
 98%|█████████▊| 49/50 [7:37:59<07:37, 457.80s/trial, best loss: -0.18523043825516827]

  0%|          | 0/139 [00:00<?, ?it/s]

  1%|          | 1/139 [00:00<00:36,  3.82it/s]

  2%|2         | 3/139 [00:00<00:27,  5.02it/s]

  5%|5         | 7/139 [00:00<00:20,  6.59it/s]

  7%|7         | 10/139 [00:00<00:17,  7.43it/s]

 12%|#1        | 16/139 [00:00<00:12, 10.08it/s]

 14%|#4        | 20/139 [00:01<00:09, 12.76it/s]

 19%|#8        | 26/139 [00:01<00:06, 16.46it/s]

 22%|##2       | 31/139 [00:01<00:05, 19.06it/s]

 25%|##5       | 35/139 [00:01<00:04, 21.32it/s]

 29%|##9       | 41/139 [00:01<00:03, 26.19it/s]

 32%|###2      | 45/139 [00:01<00:03, 27.67it/s]

 35%|###5      | 49/139 [00:01<00:03, 28.57it/s]

 38%|###8      | 53/139 [00:01<00:02, 29.86it/s]

 43%|####3     | 60/139 [00:02<00:02, 32.45it/s]

 46%|####6     | 64/139 [00:02<00:02, 28.71it/s]

 49%|####8     | 68/139 [00:02<00:02, 30.66it/s]

 52%|#####1    | 72/139 [00:02<00:02, 30.86it/s]

 57%|#####6    | 79/139 [00:02<00:01, 36.82it/s]

 60%|######    | 84/139 [00:02<00:01, 30.21it/s]

 63%|######3

Stacking columns...                                                                   
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   3.0s                
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.9s                
[Pipeline] ............. (step 3 of 3) Processing e_net, total=  18.8s                
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   2.8s                
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.8s                
[Pipeline] ............. (step 3 of 3) Processing e_net, total=  22.5s                
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   3.3s                
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   3.3s                
[Pipeline] ............. (step 3 of 3) Processing e_net, total=  15.6s                
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   3.5s                
[Pipeline] ...... (step 2 of 3) Processing 

In [34]:
pickle.dump(best_param, open(out_dir.joinpath('best_param.pkl'), 'wb'))
pickle.dump(trials, open(out_dir.joinpath('trials.pkl'), 'wb'))

In [35]:
best_param

{'C': 0.033729490904225834,
 'l1_ratio': 0.1653172093468481,
 'w': 0.5665616931869023}

# Use the selected params to do the feature selection


In [36]:
enet_params = {k: v for k, v in best_param.items() if k != 'w'}

post_extraction_pipeline = Pipeline(
    [('transformer', MeanScaledArcsinhTransformer()),
     ('feature_selection', FeatureSelector(num_features=500, min_selections=4, n_jobs=30,
                                           feature_names=to_xtract, always_keep=dmdb_feat,
                                           random_state=4))], verbose=True)

In [37]:
mg.update_w(best_param['w'])
(a, b), dwpc = piecewise_extraction(function=mg.extract_dwpc, 
                                 to_split='metapaths', block_size=block_size,
                                 axis=1,
                                 metapaths=to_xtract, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_dis),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)

Changing w from 0.7272580526994912 to 0.5665616931869023. Please wait...


100%|██████████| 21/21 [45:02<00:00, 128.68s/it]


In [38]:
post_extraction_pipeline.fit(dwpc[train_index], y[train_index])

[Pipeline] ....... (step 1 of 2) Processing transformer, total=  35.5s
Running Cor


/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Running Chi2
Running RFE
Fitting estimator with 7012 features.
Fitting estimator with 5610 features.
Fitting estimator with 4208 features.
Fitting estimator with 2806 features.
Fitting estimator with 1404 features.
Running LR
Running RF
Running XG
[Pipeline] . (step 2 of 2) Processing feature_selection, total=23.8min


Pipeline(steps=[('transformer',
                 <__main__.MeanScaledArcsinhTransformer object at 0x7f9f5ae09be0>),
                ('feature_selection',
                 FeatureSelector(always_keep={'CaBPawD', 'CaGnrBPawD',
                                              'CaGpoAawD', 'CaGpoAsoD',
                                              'CaGpoGnrBPawD', 'CaGpoGprBPawD',
                                              'CaGprBPawD', 'CaGprBPpAsoD',
                                              'CaGrBPawD', 'CawPWawD',
                                              'CinBPawD', 'CinGawD',
                                              'CinGawPpsD', 'CinGmD',
                                              'CinGmPpsD', 'CinGpoAawD',
                                              'CinGpoA...
                                                'CpoAitTcD', 'CaBPpoGpr>GawD',
                                                'CaGpoPWpoCmD', 'CawPWawPpsD',
                                                

In [39]:
pe_feats = post_extraction_pipeline[1].keep_features_

In [40]:
len(pe_feats), len(keep_features) 

(124, 139)

In [41]:
len(set(keep_features) - set(pe_feats))

43

In [42]:
feat_df = post_extraction_pipeline[1].feature_selection_df_
feat_df.sort_values('total', ascending=False).to_csv(out_dir.joinpath('feature_selection_df.csv'), index=False)

In [43]:
pickle.dump(post_extraction_pipeline[1], open(out_dir.joinpath('feature_selector.pkl'), 'wb'))
pd.Series(post_extraction_pipeline[1].keep_features_).to_csv(out_dir.joinpath('kept_features.txt'), index=False)

In [44]:
enet_params = {k: v for k, v in best_param.items() if k != 'w'}


# Set up the post feature extraction pipeline
post_extraction_pipeline = Pipeline(
    [('transformer', MeanScaledArcsinhTransformer()),
     ('maxabs_scale', MaxAbsScaler()),
     ('e_net', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100, **enet_params,
                                  random_state=rs+6))], verbose=True)

## Get the dwpc information for the current pairs
(stuff), dwpc = mg.extract_dwpc(metapaths=pe_feats, 
                                start_nodes=list(keep_comps), 
                                end_nodes=list(keep_dis),
                                return_sparse=True,
                                sparse_df=False,
                                n_jobs=30)

Preparing function arguments...
Calculating DWPCs...


100%|██████████| 124/124 [02:32<00:00,  1.23s/it]



Reshaping Result Matrices...


100%|██████████| 124/124 [00:03<00:00, 34.96it/s]


Stacking columns...


In [45]:
this_dwpc = dwpc[train_index]
this_y = y[train_index]

In [46]:
cv = cross_validate(post_extraction_pipeline, this_dwpc, this_y, cv=5, 
                    scoring=['average_precision', 'roc_auc'], return_estimator=True)

auroc_mean = cv['test_roc_auc'].mean()
avg_pre_mean = cv['test_average_precision'].mean()
auroc_std = cv['test_roc_auc'].std()
avg_pre_std = cv['test_average_precision'].std()

score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)

[Pipeline] ....... (step 1 of 3) Processing transformer, total=   2.7s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.4s
[Pipeline] ............. (step 3 of 3) Processing e_net, total= 1.2min
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   2.6s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.3s
[Pipeline] ............. (step 3 of 3) Processing e_net, total= 1.4min
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   2.9s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.7s
[Pipeline] ............. (step 3 of 3) Processing e_net, total= 1.1min
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   3.1s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   3.0s
[Pipeline] ............. (step 3 of 3) Processing e_net, total= 1.2min
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   2.8s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.5s


/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[Pipeline] ............. (step 3 of 3) Processing e_net, total= 1.5min


In [47]:
print(('AUROC Scores: '+5*'{:1.4f}, ').format(*cv['test_roc_auc'].tolist()))
print('Mean AUROC: {:1.4f}'.format(auroc_mean))
print(('Avg Pre. Scores: '+5*'{:1.4f}, ').format(*cv['test_average_precision'].tolist()))
print('Mean Avg Pre: {:1.4f}'.format(avg_pre_mean))
print('STD AUROC: {:1.4f}'.format(auroc_std))
print('STD Avg Pre: {:1.4f}'.format(avg_pre_std))
print('Loss: {:1.4f}'.format(score))

AUROC Scores: 0.7293, 0.7219, 0.7768, 0.7300, 0.5955, 
Mean AUROC: 0.7107
Avg Pre. Scores: 0.0783, 0.0578, 0.1054, 0.1383, 0.0478, 
Mean Avg Pre: 0.0855
STD AUROC: 0.0608
STD Avg Pre: 0.0329
Loss: -0.1868
